In [15]:
import pandas as pd
import requests
import datetime
import numpy as np
import altair as alt
import datapane as dp

In [16]:
# inputs

dp.Params.load_defaults('datapane.yaml')
CURRENT_CASH_GBP = dp.Params.get('current_cash', 100000)
FORECAST_LENGTH_DAYS = 365 * dp.Params.get('forecast_years', 2)
WEEKLY_GROWTH_PERCENT = dp.Params.get('weekly_growth_rate', 2)
INITIAL_REVENUE = dp.Params.get('initial_revenue', 100)
MONTHLY_BURN = dp.Params.get('monthly_burn', CURRENT_CASH_GBP / 12)

# Misc cash injection

INJECTION_AMOUNT = dp.Params.get('cash_injection', 0) # TODO: calculate as derived value
INJECTION_OFFSET = dp.Params.get('cash_injection_offset', 0) # Number of mos until R&D grant

# derived values

monthly_growth_rate = pow((1+WEEKLY_GROWTH_PERCENT/100), 4) - 1
today = pd.to_datetime('today')
end_forecast = today + pd.Timedelta(FORECAST_LENGTH_DAYS, unit='d')

burn = MONTHLY_BURN

In [17]:
CURRENT_CASH_GBP

100000

In [3]:
# This forecast is worst-case scenario / most conservative - predicated on if you do not grow at all.

df = pd.DataFrame(columns=[])

idx = pd.date_range(today, end_forecast, freq='m')
df = df.reindex(idx, fill_value=0)
df.index.names = ['date']
df.reset_index(inplace=True)

df['burn'] = burn
df['revenue_0g'] = INITIAL_REVENUE
df['injection'] = 0

# Set initial cash and revenue
df.loc[0, 'cash_0g'] = CURRENT_CASH_GBP
df.loc[0, 'cash_dg'] = CURRENT_CASH_GBP
df.loc[0, 'revenue_dg'] = INITIAL_REVENUE

df.loc[INJECTION_OFFSET + 1, 'injection'] = INJECTION_AMOUNT

# Cas injection (e.g. R&D tax or fundraise)
df.loc[INJECTION_OFFSET + 1, 'injection'] = INJECTION_AMOUNT # 0dx adjustment

for i in range(1, len(df)):
    prev = df.loc[i-1]
    df.loc[i, 'cash_0g'] = prev['cash_0g'] - prev['burn'] + prev['injection'] + prev['revenue_0g']
    df.loc[i, 'cash_dg'] = prev['cash_dg'] - prev['burn'] + prev['injection'] + prev['revenue_dg']
    df.loc[i, 'revenue_dg'] = prev['revenue_dg'] * (1 + monthly_growth_rate)
df.dtypes

date          datetime64[ns]
burn                 float64
revenue_0g             int64
injection              int64
cash_0g              float64
cash_dg              float64
revenue_dg           float64
dtype: object

In [4]:
# This is a variable forecast for modelling different growth-rates.
# Initial growth rate is set to be the current growth rate

# This cannot be 0, as growth rate wouldn't work
start_revenue = INITIAL_REVENUE if INITIAL_REVENUE > 100 else 100
scale = (-CURRENT_CASH_GBP, CURRENT_CASH_GBP * 2)



In [5]:
# This is a variable forecast for modelling different growth-rates.
# Initial growth rate is set to be the current growth rate

# This cannot be 0, as growth rate wouldn't work
start_revenue = INITIAL_REVENUE if INITIAL_REVENUE > 100 else 100
scale = (-CURRENT_CASH_GBP, CURRENT_CASH_GBP * 2)

monthly_growth_percent = WEEKLY_GROWTH_PERCENT * 4
slider = alt.binding_range(min=0, max=100, step=0.1, name='Monthly growth rate (%)')
selector = alt.selection_single(fields=['month_growth'], bind=slider, init={'month_growth': monthly_growth_percent})

base = alt.Chart(df[['date', 'cash_0g']]).transform_window(
    # Use count() to find out the current month we're on (i.e. the index), as we use that as the power for our growth rate
    index='count()'
).transform_calculate(
    # Transform a % growth rate (i.e. 7) into sometime useful (i.e. 1.07).
    # Raise that to our current month (i.e. for month 2, 1.07 ** 2 = ~1.145), and multiply that by our revenue amount (~1.145 * 500)
    forecast_revenue = ((1+(selector.month_growth / 100)) ** alt.datum.index) * start_revenue
).transform_window(
    # Take what is in practice the cumulative product of revenue (i.e. total revenue earned so far)
    forecast_total_revenue = 'sum(forecast_revenue)', frame=[None, 0]
).transform_calculate(
    # Add cumprod to current cash to find cash standing (i.e. if we have £100k cash, and 10 months of £10k revenue, we have £200k now)
    adj_cash = alt.datum.cash_0g + alt.datum.forecast_total_revenue
).encode(x='yearmonth(date):T')

cash = base.encode(y=alt.Y('adj_cash:Q', scale=alt.Scale(domain=scale))).mark_area(opacity=0.75)
revenue = base.encode(y='forecast_total_revenue:Q').mark_line(color='#f26522', size=5) 
revenue_growth = base.encode(y='forecast_revenue:Q').mark_line(color='purple', size=5)

charts = cash + revenue + revenue_growth

chart_final = charts.add_selection(selector).interactive().properties(width=850)
chart_final


alt.LayerChart(...)

In [6]:
dead_date_0g = df[df['cash_0g'] < 25000].iloc[0].date
months_remaining_0g = dead_date_0g.to_period('M') - today.to_period('M')


In [7]:
dead_date_0g.to_period('M')

Period('2021-04', 'M')

In [8]:
dead_date_dg = df[df['cash_dg'] < 25000].iloc[0].date
months_remaining_dg = dead_date_dg.to_period('M') - today.to_period('M')


In [9]:
from datapane import Markdown, Plot, Table, Report

report = [
    Markdown(f"""
# Startup Finance Report
Date: {today.strftime("%B %d, %Y")}

Your burn is currently **£{round(burn)}** per month, and you have **£{CURRENT_CASH_GBP}** in the bank.

> This includes an presumed cash injection of {INJECTION_AMOUNT} on month {INJECTION_OFFSET}.

## Cash forecast
#### Worst case: 0 growth scenario
If you don't grow at all, you have **{months_remaining_0g.n}** months remaining, and will run out of money around {dead_date_0g.strftime("%B %Y")}.

#### Default case: continued growth scenario
If you continue to grow at your current rate, you have **{months_remaining_dg.n}** months remaining, and will run out of money around {dead_date_dg.strftime("%B %Y")}.

> In this dataset, `cash_0g`/`revenue_0g` presumes no further growth, whereas `cash_dg`/`revenue_dg` presumes growth at the current rate continues. 
"""),
Table(df),
Markdown("""

## Modelling growth scenarios

This plot models cash and revenue, dependent on various other growth scenarios and based on this month's burn. 

- Blue area is cash remaining
- Orange line is the cumulative sum of all revenue
- Purple line is monthly revenue

> Drag the slider to adjust growth rate. The growth rate at which the blue area never crosses 0 is the growth you need to achieve **[Default Alive](http://paulgraham.com/aord.html)**.

"""),
    Plot(chart_final)
]

In [10]:
Report(*report).preview()

In [11]:
Report(*report).publish(headline='Startup Finance Report', name = 'startup_finance_report')

15:06:25 [INFO ] Uploading assets for Report
15:06:25 [INFO ] Uploading /home/khuyentran/gallery/startup-calculator/dp-tmp-6oyj44gf/dp-tmp-56hutwon.arrow
15:06:27 [INFO ] Uploading /home/khuyentran/gallery/startup-calculator/dp-tmp-6oyj44gf/dp-tmp-7_z8rh1n.vl.json
15:06:28 [INFO ] Report published to Datapane as https://datapane.com/khuyen/reports/startup_finance_report_523fc74c/


In [14]:
dp.Report(
dp.Plot(chart_final)).publish(headline='Startup Growth Calculator', name = 'startup_graph')

15:40:35 [INFO ] Uploading assets for Report
15:40:36 [INFO ] Uploading /home/khuyentran/gallery/startup-calculator/dp-tmp-6oyj44gf/dp-tmp-u_u55k3j.vl.json
15:40:37 [INFO ] Report published to Datapane as https://datapane.com/khuyen/reports/startup_graph/
